In [1]:
import numpy as np
import pandas as pd
from sqlalchemy import create_engine
import requests
#from config import api_key
import time
#from us import states
from scipy.stats import linregress
from matplotlib import pyplot as plt
from fredapi import Fred
import datetime as dt
fred = Fred(api_key='ADD YOUR API KEY')

In [2]:
#STEP 1 CSV - reading the CSV
inf_path="inflation.csv"

inflation = pd.read_csv(inf_path)

INF = pd.DataFrame(inflation)

In [3]:
#STEP 2 CSV - Rename CSV Date
INF_df = INF.rename(columns={'Date': 'qtr_dt'})
INF_df

,qtr_dt,INF
0,1947-01-01,18.87
1,1947-04-01,18.33
2,1947-07-01,12.07
3,1947-10-01,9.30
4,1948-01-01,8.77
...,...,...
287,2018-10-01,2.20
288,2019-01-01,1.67
289,2019-04-01,1.80
290,2019-07-01,1.73


In [4]:
#STEP 3 CSV - Convert qtr_dt to datetime
INF_df['qtr_dt'] = pd.to_datetime(INF_df['qtr_dt'])
INF_df

,qtr_dt,INF
0,1947-01-01,18.87
1,1947-04-01,18.33
2,1947-07-01,12.07
3,1947-10-01,9.30
4,1948-01-01,8.77
...,...,...
287,2018-10-01,2.20
288,2019-01-01,1.67
289,2019-04-01,1.80
290,2019-07-01,1.73


In [5]:
#STEP 1 Retrieve Unemployment data from FRED api
UNRATE = {}
UNRATE['UNEM'] = fred.get_series('UNRATE',frequency = 'q')
UNRATE_df = pd.DataFrame(UNRATE)
UNRATE_df

,UNEM
1948-01-01,3.7
1948-04-01,3.7
1948-07-01,3.8
1948-10-01,3.8
1949-01-01,4.7
...,...
2019-07-01,3.6
2019-10-01,3.5
2020-01-01,3.8
2020-04-01,13.0


In [6]:
#STEP 2 Unemployment - Rename the index
UNRATE_df.index.name = 'qtr_dt'
UNRATE_df

UNRATE_df= UNRATE_df.reset_index()
UNRATE_df

,qtr_dt,UNEM
0,1948-01-01,3.7
1,1948-04-01,3.7
2,1948-07-01,3.8
3,1948-10-01,3.8
4,1949-01-01,4.7
...,...,...
286,2019-07-01,3.6
287,2019-10-01,3.5
288,2020-01-01,3.8
289,2020-04-01,13.0


In [7]:
#Merge final dataframe for US 
final_df= pd.merge(UNRATE_df, INF_df, on = 'qtr_dt')
final_df

,qtr_dt,UNEM,INF
0,1948-01-01,3.7,8.77
1,1948-04-01,3.7,9.10
2,1948-07-01,3.8,8.43
3,1948-10-01,3.8,4.63
4,1949-01-01,4.7,1.43
...,...,...,...
283,2018-10-01,3.8,2.20
284,2019-01-01,3.9,1.67
285,2019-04-01,3.6,1.80
286,2019-07-01,3.6,1.73


**Connect to Local Database**

In [8]:
rds_connection_string = "kanika:postgres@localhost:5432/phillips_db"
engine = create_engine(f'postgresql://{rds_connection_string}')

**Check for Tables**

In [14]:
engine.table_names()

['phillips_curve']

**Use pandas to load merged DataFrame into database**

In [15]:
final_df.to_sql(name='phillips_curve', con=engine, if_exists='append', index=False)

ProgrammingError: (psycopg2.errors.UndefinedColumn) column "UNEM" of relation "phillips_curve" does not exist
LINE 1: INSERT INTO phillips_curve (qtr_dt, "UNEM", "INF") VALUES ('...
                                            ^

[SQL: INSERT INTO phillips_curve (qtr_dt, "UNEM", "INF") VALUES (%(qtr_dt)s, %(UNEM)s, %(INF)s)]
[parameters: ({'qtr_dt': datetime.datetime(1948, 1, 1, 0, 0), 'UNEM': 3.7, 'INF': 8.77}, {'qtr_dt': datetime.datetime(1948, 4, 1, 0, 0), 'UNEM': 3.7, 'INF': 9.1}, {'qtr_dt': datetime.datetime(1948, 7, 1, 0, 0), 'UNEM': 3.8, 'INF': 8.43}, {'qtr_dt': datetime.datetime(1948, 10, 1, 0, 0), 'UNEM': 3.8, 'INF': 4.63}, {'qtr_dt': datetime.datetime(1949, 1, 1, 0, 0), 'UNEM': 4.7, 'INF': 1.43}, {'qtr_dt': datetime.datetime(1949, 4, 1, 0, 0), 'UNEM': 5.9, 'INF': -0.27}, {'qtr_dt': datetime.datetime(1949, 7, 1, 0, 0), 'UNEM': 6.7, 'INF': -2.73}, {'qtr_dt': datetime.datetime(1949, 10, 1, 0, 0), 'UNEM': 7.0, 'INF': -2.23}  ... displaying 10 of 288 total bound parameter sets ... 

**Confirm data has been added by querying the customer_name table**

***NOTE: can also check using pgAdmin***

In [13]:
pd.read_sql_query('select * from phillips_curve', con=engine).head()

,qtr_dt,unem,inf
